![](images/convolutions.png)

(the four images on the right are derived from http://intellabs.github.io/RiverTrail/tutorial/ and http://docs.gimp.org/en/plug-in-convmatrix.html)

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
import tensorflow as tf

# Clear the graph because we might run this
# cell in Jupyter many times, and we don't
# want multiple copies of the graph.
tf.python.framework.ops.reset_default_graph()

inputs = tf.placeholder(tf.float32, [None, 784], name="inputs")
expected_outputs = tf.placeholder(tf.float32, [None, 10], name="expected_outputs")

inputs_4d = tf.reshape(inputs, [-1, 28, 28, 1])

conv1_kernel = tf.Variable(
    tf.random_normal([3, 3, 1, 8], stddev=0.05), tf.float32, name='conv1_kernel')
conv1_biases = tf.Variable(
    tf.random_normal([8], stddev=0.05), tf.float32, name="conv1_biases")

conv2_kernel = tf.Variable(
    tf.random_normal([3, 3, 8, 16], stddev=0.05), tf.float32, name='conv2_kernel')
conv2_biases = tf.Variable(
    tf.random_normal([16], stddev=0.05), tf.float32, name="conv2_biases")

weights1 = tf.Variable(
    tf.random_normal([28/4*28/4*16, 100], stddev=0.05), tf.float32, name="weights1")
biases1 = tf.Variable(
    tf.random_normal([100], stddev=0.05), tf.float32, name="biases1")

weights2 = tf.Variable(
    tf.random_normal([100, 50], stddev=0.05), tf.float32, name="weights2")
biases2 = tf.Variable(
    tf.random_normal([50], stddev=0.05), tf.float32, name="biases2")

weights3 = tf.Variable(tf.zeros([50, 10], tf.float32), name="weights3")
biases3 = tf.Variable(tf.zeros([10]), tf.float32, name="biases3")


with tf.name_scope("convoluted1"):
    convoluted1 = tf.nn.relu(tf.nn.conv2d(inputs_4d, conv1_kernel, strides=[1, 2, 2, 1], padding='SAME') + conv1_biases)
    
with tf.name_scope("convoluted2"):
    convoluted2 = tf.nn.relu(tf.nn.conv2d(convoluted1, conv2_kernel, strides=[1, 2, 2, 1], padding='SAME') + conv2_biases)
    convoluted2_2d = tf.reshape(convoluted2, [-1, 28/4*28/4*16])
    
with tf.name_scope("layer1"):
    layer1_outputs = tf.nn.relu(tf.matmul(convoluted2_2d, weights1) + biases1)

with tf.name_scope("layer2"):
    layer2_outputs = tf.nn.relu(tf.matmul(layer1_outputs, weights2) + biases2)

with tf.name_scope("softmax_layer"):
    outputs = tf.nn.softmax(tf.matmul(layer2_outputs, weights3) + biases3)
    
with tf.name_scope("cross_entropy"):
    cross_entropy = -tf.reduce_sum(expected_outputs*tf.log(outputs))

with tf.name_scope("train_step"):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    train_step = optimizer.minimize(cross_entropy)

initialize = tf.initialize_all_variables()

with tf.name_scope("accuracy"):
    labels = tf.argmax(outputs, 1)
    expected_labels = tf.argmax(expected_outputs, 1)
    correct_prediction = tf.equal(labels, expected_labels)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as session:
    session.run(initialize)
    for step in xrange(10000):
        batch_inputs, batch_outputs = mnist.train.next_batch(100)
        session.run(train_step, {
                inputs: batch_inputs,
                expected_outputs: batch_outputs
            })

        if step % 100 == 0:
            print session.run(accuracy, {
                    inputs: mnist.test.images,
                    expected_outputs: mnist.test.labels
                })
